# Word3Vec (11th November 2021)

This notebook applies word2vec to the corpus of tribunal decisions.

In particular, the notebook does:

1. Data preparation for word2vec.

2. Implementation of word2vec.

3. Topic model with Latent Dirichlet Allocation (LDA) and Latent Semantic Indexing (LSI).

4. Key word extraction

The resulting trained model is... .

This notebook should run in the tfm environment, which can be created with the environment.yml file.

In [3]:
from os import listdir
from os.path import isfile, join, getsize
import numpy as np

import time
import re
import json
import pickle
import pandas as pd
import whois
import sys
import datetime
from tqdm import tqdm
import textract
import gensim
import spacy
import scipy as sp
import sys
import multiprocessing
import gensim
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import remove_stopwords

import logging

from smart_open import smart_open
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer


import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

from collections import defaultdict  # For word frequency


# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)



IN_COLAB = 'google.colab' in sys.modules


# What environment am I using?
print(f'Current environment: {sys.executable}')

# Change the current working directory
os.chdir('/Users/albertamurgopacheco/Documents/GitHub/TFM')
# What's my working directory?
print(f'Current working directory: {os.getcwd()}')


Current environment: /Users/albertamurgopacheco/anaconda3/envs/tfm/bin/python
Current working directory: /Users/albertamurgopacheco/Documents/GitHub/TFM


/Users/albertamurgopacheco/anaconda3/envs/tfm/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [11]:
# Define working directories in colab and local execution

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    docs_path = '/content/gdrive/MyDrive/TFM/data/raw'
    input_path = '/content/gdrive/MyDrive/TFM'
    output_path = '/content/gdrive/MyDrive/TFM/output'

else:
    docs_path = './data/raw'
    input_path = '.'
    output_path = './output'

# WORD2VEC

# 1. The data preparation

Loading the data to a list of documents (corpus) where each document is a judicial decision.

In [3]:
# Open jsonDataFinal file as data
with open('./data/jsonDataFinal.json') as json_file:
    data = json.load(json_file)

# List to store the judicial decisions
corpus = []

corruptFiles = ['HU077022015', 'HU029682017']

# Search data list of dictionaries for dict where {"File":} = file_name
for d in tqdm(data):
    # Dealing with corrupt and empty files
    if d.get('File') not in corruptFiles:
        doc = d.get('String')
        if doc:
            corpus.append(doc)
        else:
            continue

print(f'The corpus includes {len(corpus)} documents.')
print(f'The documents are {type(corpus[0])}.')

100%|██████████| 35305/35305 [00:00<00:00, 210756.30it/s]

The corpus includes 35305 documents.
The documents are <class 'str'>.


Cleaning each document

In [4]:
# Gensim-implemented filters for preprocessing data
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, 
strip_multiple_whitespaces, strip_non_alphanum, strip_numeric, remove_stopwords]

# List storing thr preprocessed documents
corpus_clean = [preprocess_string(doc, CUSTOM_FILTERS) for doc in corpus]

# Removing non-numerical characters
#brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in corpus_clean)

In [5]:
# Number of available processing cores
cores = multiprocessing.cpu_count()
print(f'Available cores {cores}.')

Available cores 10.


Lemmatizing and removing the stopwords and non-alphabetic characters.

In [13]:
# IF RAN, DO IT BEFORE CLEANING... 

import spacy
#nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
#nlp = spacy.load('en_core_web_sm') # disabling Named Entity Recognition for speed
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

def stem_doc(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    stem_doc = [porter_stemmer.stem(wd) for wd in doc]
    # if len(wd) > 2
    return stem_doc

    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small

#txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size = 50, n_process = cores -1)]
stemmed_corpus = [stem_doc(doc) for doc in corpus_clean]

#print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

TypeError: 'module' object is not callable

In [15]:
print(stemmed_corpus[0][:15])

['pic', 'iac', 'fh', 'ck', 'v', 'upper', 'tribun', 'immigr', 'asylum', 'chamber', 'appeal', 'number', 'hu', 'immigr', 'act']


In [16]:
wordnet_lemmatizer = WordNetLemmatizer()

# Word2Vec uses context words to learn the vector representation of a target word,
# if a sentence is only one or two words long,
# the benefit for the training is very small

def lemmatizer_doc(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    lemm_doc = [wordnet_lemmatizer.lemmatize(wd) for wd in doc]
    # if len(wd) > 2
    return lemm_doc



# txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size = 50, n_process = cores -1)]
lemmatized_corpus = [lemmatizer_doc(doc) for doc in stemmed_corpus]

In [17]:
#lemmatized_corpus = lemmatized
print(lemmatized_corpus[0][:15])

['pic', 'iac', 'fh', 'ck', 'v', 'upper', 'tribun', 'immigr', 'asylum', 'chamber', 'appeal', 'number', 'hu', 'immigr', 'act']


Detecting common phrases (multi-word or collocations) expressions from the stream of sentences.

In [18]:
sent = [row.split() for row in corpus if row]

phrases = Phrases(sent, min_count = 30, progress_per = 10000)

In [19]:
bigram = Phraser(phrases)

#sentences = bigram[corpus_clean]
sentences = bigram[lemmatized_corpus]

word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

90189

In [53]:
sorted(word_freq, key = word_freq.get, reverse = True)[:20]

NameError: name 'word_freq' is not defined

# 2. Implementation of word2vec model

The word embedding model is a model that can provide numerical vectors for a given word.

In [13]:
import multiprocessing

from gensim.models import Word2Vec

w2v_model = Word2Vec(min_count = 20,
                     window = 10,
                     vector_size = 300,
                     sample = 6e-5,
                     alpha = 0.03,
                     min_alpha = 0.0007,
                     negative = 20,
                     workers = cores-1)

In [24]:
#t = time()

w2v_model.build_vocab(sentences, progress_per = 10000)

#print('Time to build the vocabulary: {} mins'.format(round((time() - t) / 60, 2)))

In [2]:
#t = time()

w2v_model.train(sentences, total_examples = w2v_model.corpus_count, epochs = 30, report_delay = 1)

#print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

NameError: name 'w2v_model' is not defined

In [1]:
w2v_model.save('./output/gensim-model-All_bigrams_lemmatized')

NameError: name 'w2v_model' is not defined

In [20]:
model_lemm = Word2Vec.load('./output/gensim-model-All_bigrams_lemmatized')
model = Word2Vec.load('./output/gensim-model-All_bigrams')

Similarities:

In [22]:

model.wv.similarity('refugee', 'error')

-0.051099308

In [29]:
model_lemm.wv.similarity('refuge', 'convent')
model_lemm.wv.similarity('refuge', 'geneva')

0.5783935

Most similar:

In [30]:
word = 'geneva'

try:
    vec = model_lemm.wv.most_similar(positive = [word])
    print(vec)
except KeyError:
    print(f'The word {word} does not appear in this model')
    


[('refuge', 0.6041285991668701), ('convent', 0.551478922367096), ('stateless_person', 0.45458465814590454), ('protocol', 0.41364872455596924), ('humanitarian', 0.3975497782230377), ('refoul', 0.3917050361633301), ('european', 0.3764244616031647), ('derog', 0.3744756877422333), ('defin', 0.3737797439098358), ('qualif', 0.3719629943370819)]


In [32]:
model.wv.most_similar(positive = ["refugee"])

[('refugee_status', 0.6638965010643005),
 ('refugee_convention', 0.5883638262748718),
 ('refugees', 0.5871933698654175),
 ('geneva', 0.5129669308662415),
 ('granted_refugee', 0.5052950382232666),
 ('humanitarian_protection', 0.49545037746429443),
 ('recognised', 0.4840916395187378),
 ('international_protection', 0.48296958208084106),
 ('founded_fear', 0.4610556662082672),
 ('cessation_clause', 0.4556953012943268)]

In [40]:
model.wv.most_similar(positive=["biometric"])
model_lemm.wv.most_similar(positive=["eurodac"])

[('fingerprint_match', 0.6826852560043335),
 ('fingerprint', 0.5869845151901245),
 ('ifb', 0.5495399832725525),
 ('data', 0.3474268317222595),
 ('yi', 0.34197482466697693),
 ('dunkirk', 0.3346039056777954),
 ('hungari', 0.3305904269218445),
 ('lookup_tool', 0.3303433656692505),
 ('match', 0.32926681637763977),
 ('printout', 0.31014716625213623)]

In [41]:
model.wv.most_similar(positive=["eurodac"])


[('fingerprint_match', 0.7210105061531067),
 ('fingerprint', 0.6880550980567932),
 ('fingerprints', 0.5672035217285156),
 ('ifb', 0.5238295197486877),
 ('fingerprinted', 0.49927783012390137),
 ('prints', 0.47691816091537476),
 ('asu', 0.40677061676979065),
 ('yi', 0.3775590658187866),
 ('fingerprinting', 0.3729415237903595),
 ('hungary', 0.3618243634700775)]

End of gensim tutorial

In [42]:
model.wv.most_similar(positive=["uk"])

[('kingdom', 0.7254781723022461),
 ('united', 0.7136476039886475),
 ('lawfully', 0.5895837545394897),
 ('remain', 0.5732899308204651),
 ('years', 0.5701707005500793),
 ('leave', 0.5506826639175415),
 ('lived', 0.5331628322601318),
 ('aged', 0.5244055390357971),
 ('overstayed', 0.5227060914039612),
 ('parents', 0.5147008895874023)]

Odd-One-Out:

In [45]:
model.wv.doesnt_match(['refuggee', 'uk', 'remain'])

'remain'

In [52]:
model.wv.doesnt_match(['biometric', 'eurodac', 'identity'])

print(f"Word 'eurodac' appeared {model.wv.get_vecattr('eurodac', 'count')} times in the training corpus.")
print(f"Word 'biometric' appeared {model.wv.get_vecattr('biometric', 'count')} times in the training corpus.")


Word 'eurodac' appeared 241 times in the training corpus.
Word 'biometric' appeared 345 times in the training corpus.


In [34]:
data = [d for d in dataset]
print(type(data[0][0]))

<class 'str'>


In [15]:
# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in corpus_pre]

# Create dictionary
dictionary = corpora.Dictionary(texts)

# Get information about the dictionary
print(dictionary)

Dictionary(1613 unique tokens: ['abandoned', 'ability', 'able', 'absence', 'accept']...)


In [17]:
# Show the word to id map
#print(dictionary.token2id)


create a bag of words corpus

In [18]:
# Tokenize the docs
tokenized_list = [simple_preprocess(doc) for doc in corpus_pre]

# Create the Corpus
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]
#pprint(mycorpus)

word_counts = [[(mydict[id], count) for id, count in line] for line in mycorpus]
#pprint(word_counts)

In [12]:
w2v_model.wv.doesnt_match(['jimbo', 'milhouse', 'kearney'])

# 3. Topic model with Latent Dirichlet Allocation (LDA) and Latent Semantic Indexing (LSI)

The objective of topic models is to extract the underlying topics from a collection of text documents. Each document in the text is considered as a combination of topics and each topic is considered as a combination of related words.

Topic modeling can be done by algorithms like Latent Dirichlet Allocation (LDA) and Latent Semantic Indexing (LSI).

3.1. Latent Dirichlet Allocation (LDA)
Each document can be described by a distribution of topics and each topic can be described by a distribution of words. https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

In [ ]:
from gensim.models import LdaModel, LdaMulticore

In [83]:
# Starting from corpus_clean

# Build the bigram and trigram models
bigram = gensim.models.Phrases(corpus_clean, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus_clean], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



NameError: name 'data_words' is not defined

In [85]:
# See trigram example
#print(trigram_mod[bigram_mod[corpus_clean[0]]])

In [86]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [88]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Remove Stop Words
data_words_nostops = remove_stopwords(corpus_clean)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])



[['pic', 'upper', 'tribunal', 'immigration', 'asylum', 'chamber', 'appeal', 'number', 'immigration', 'act', 'hear', 'field_house', 'decision', 'reason', 'promulgate', 'extempore', 'upper', 'tribunal', 'judge', 'anonymity', 'direction', 'appellant', 'respondent', 'counsel_instructe', 'consultant', 'office', 'present', 'officer', 'decision', 'reason', 'appellant', 'bear', 'section', 'nationality', 'act', 'decision', 'secretary', 'state', 'refuse', 'application', 'leave', 'remain', 'appeal', 'decision', 'hear', 'tier', 'tribunal', 'march', 'reason', 'set', 'decision', 'refuse', 'reason', 'set', 'decision', 'copy', 'annex', 'decision', 'set', 'aside', 'appellant', 'enter', 'visit', 'visa', 'remain', 'leave', 'form', 'relationship', 'refer', 'sponsor', 'son', 'british', 'previous', 'relationship', 'case', 'long', 'contact', 'child', 'father', 'appellant', 'form', 'parental', 'relationship', 'appellant', 'family', 'live', 'relatively', 'recently', 'sponsor', 'employ', 'secretary', 'state', '

In [89]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corp = [id2word.doc2bow(text) for text in texts]

# View
print(corp[:1])

[[(0, 1), (1, 1), (2, 3), (3, 1), (4, 9), (5, 2), (6, 1), (7, 3), (8, 8), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 1), (16, 1), (17, 3), (18, 1), (19, 1), (20, 3), (21, 22), (22, 33), (23, 6), (24, 5), (25, 1), (26, 1), (27, 1), (28, 1), (29, 5), (30, 4), (31, 1), (32, 1), (33, 2), (34, 1), (35, 2), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 10), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 2), (50, 1), (51, 19), (52, 1), (53, 1), (54, 2), (55, 1), (56, 2), (57, 10), (58, 5), (59, 3), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 2), (71, 1), (72, 1), (73, 7), (74, 1), (75, 2), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 4), (82, 1), (83, 1), (84, 1), (85, 1), (86, 4), (87, 5), (88, 2), (89, 1), (90, 30), (91, 1), (92, 3), (93, 1), (94, 2), (95, 1), (96, 10), (97, 1), (98, 3), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 2), (109, 2), (

In [90]:
id2word[0]

'abandon'

In [91]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corp[:1]]

[[('abandon', 1),
  ('ability', 1),
  ('able', 3),
  ('absence', 1),
  ('accept', 9),
  ('accompany', 2),
  ('accordance', 1),
  ('account', 3),
  ('act', 8),
  ('adduce', 1),
  ('ade', 1),
  ('adio', 1),
  ('adopt', 2),
  ('adverse', 1),
  ('advice', 2),
  ('age', 1),
  ('agree', 1),
  ('allow', 3),
  ('amount', 1),
  ('annex', 1),
  ('anonymity', 3),
  ('appeal', 22),
  ('appellant', 33),
  ('application', 6),
  ('apply', 5),
  ('applying', 1),
  ('approach', 1),
  ('appropriate', 1),
  ('approve', 1),
  ('article', 5),
  ('aside', 4),
  ('ask', 1),
  ('assertion', 1),
  ('assessment', 2),
  ('assistance', 1),
  ('asylum', 2),
  ('attach', 3),
  ('attract', 1),
  ('away', 1),
  ('balance', 1),
  ('balance_probabilitie', 1),
  ('bangladesh', 1),
  ('basis', 10),
  ('bear', 1),
  ('bearing_mind', 1),
  ('behalf', 1),
  ('benefit', 1),
  ('best_interest', 1),
  ('biological', 2),
  ('british', 2),
  ('capacity', 1),
  ('case', 19),
  ('cease', 1),
  ('certain', 1),
  ('certainly', 2),
 

In [108]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus = corp,
                                           id2word = id2word,
                                           num_topics = 5, 
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = 'auto',
                                           per_word_topics = True)

In [109]:


# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corp]

[(0, '0.022*"sponsor" + 0.019*"regulation" + 0.018*"marriage" + 0.016*"right" + 0.016*"appellant" + 0.015*"evidence" + 0.014*"member" + 0.013*"family" + 0.012*"residence" + 0.012*"entry_clearance"'), (1, '0.020*"decision" + 0.019*"appeal" + 0.018*"application" + 0.016*"state" + 0.015*"leave" + 0.013*"immigration" + 0.013*"case" + 0.013*"rule" + 0.012*"secretary" + 0.011*"claimant"'), (2, '0.016*"appellant" + 0.013*"evidence" + 0.011*"report" + 0.010*"risk" + 0.010*"say" + 0.009*"return" + 0.008*"adjudicator" + 0.007*"country" + 0.007*"claim" + 0.007*"case"'), (3, '0.024*"article" + 0.019*"appellant" + 0.017*"child" + 0.016*"family" + 0.015*"case" + 0.012*"right" + 0.012*"life" + 0.010*"decision" + 0.010*"immigration" + 0.009*"consider"'), (4, '0.061*"appellant" + 0.047*"appeal" + 0.036*"tribunal" + 0.034*"judge" + 0.023*"decision" + 0.022*"determination" + 0.017*"evidence" + 0.014*"respondent" + 0.014*"immigration" + 0.014*"reason"')]


In [100]:
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [110]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corp))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model = lda_model, texts = data_lemmatized, dictionary = id2word, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.931772735559883

Coherence Score:  0.3940589043683872


In [111]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corp, id2word)
vis

/Users/albertamurgopacheco/anaconda3/envs/tfm/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/albertamurgopacheco/anaconda3/envs/tfm/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/albertamurgopacheco/anaconda3/envs/tfm/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/albertamurgopacheco/anaconda3/envs/tfm/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternat

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.184303 -0.029563       1        1  34.762640
2      0.059525  0.180654       2        1  23.915208
3      0.014296  0.029241       3        1  21.803026
1     -0.056499 -0.060373       4        1  11.675673
0      0.166980 -0.119960       5        1   7.843453, topic_info=            Term          Freq         Total Category  logprob  loglift
181        judge  4.079160e+05  4.079160e+05  Default  30.0000  30.0000
21        appeal  6.914230e+05  6.914230e+05  Default  29.0000  29.0000
22     appellant  1.011411e+06  1.011411e+06  Default  28.0000  28.0000
342     tribunal  5.057810e+05  5.057810e+05  Default  27.0000  27.0000
90      decision  4.284270e+05  4.284270e+05  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
86          date  1.640699e+04  1.431997e+05   Topic5  -5.0661   0.3790
171  immigration  1.855624e+04  3.000635e+05   Topic5  -4.9430  -0.2377
364         year  1.387379e+04  9.069609e+04   Topic5  -5.2338   0.6680
299          say  1.413447e+04  1.582307e+05   Topic5  -5.2152   0.1301
21        appeal  1.446165e+04  6.914230e+05   Topic5  -5.1923  -1.3218

[459 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
14620      4  0.998681    acca
4          1  0.437076  accept
4          2  0.266385  accept
4          3  0.184995  accept
4          4  0.034281  accept
...      ...       ...     ...
364        1  0.001907    year
364        2  0.210230    year
364        3  0.562593    year
364        4  0.072307    year
364        5  0.152972    year

[936 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 2, 1])

Building LDA Mallet Model

In [113]:
# Building LDA Mallet Model
from gensim.models.wrappers import LdaMallet

# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = './data/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus = corp, num_topics = 5, id2word = id2word)

ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

Optimal number of topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

What topic each document?

In [114]:
def format_topics_sentences(ldamodel=lda_model, corpus=corp, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

NameError: name 'optimal_model' is not defined

Most representative document for each topic

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics